In [1]:
from lib.utils import generate_mean_ensemble_metrics, aggregate_pred_dataframe, generate_mean_ensemble_metrics_auto
import os
import pandas as pd
test_csv_fliles = [file for file in os.listdir('result_cv/esm-33-gearnet-ensemble/fold_0/intermediate') if 'test' in file]
test_csv_fliles.sort()
test_csv_fliles = [f'result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/{file}' for file in test_csv_fliles]

valid_csv_fliles = [file for file in os.listdir('result_cv/esm-33-gearnet-ensemble/fold_0/intermediate') if 'valid' in file]
valid_csv_fliles.sort()
valid_csv_fliles = [f'result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/{file}' for file in valid_csv_fliles]


In [2]:
agg = pd.DataFrame()
for csv_file in test_csv_fliles:
    df = aggregate_pred_dataframe([csv_file])
    me = generate_mean_ensemble_metrics(df)
    print(f'Processing {csv_file}, aggregated metrics: {me["mcc"]})')
    agg = pd.concat([agg, pd.DataFrame([me])], ignore_index=True)

agg.aggregate(['mean', 'std'])


Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_0_test.csv, aggregated metrics: 0.6566)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_10_test.csv, aggregated metrics: 0.6703)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_11_test.csv, aggregated metrics: 0.663)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_12_test.csv, aggregated metrics: 0.6566)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_13_test.csv, aggregated metrics: 0.6557)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_14_test.csv, aggregated metrics: 0.6568)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_15_test.csv, aggregated metrics: 0.6707)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_16_test.csv, aggregated metrics: 0.667)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_17_test.csv, aggregated metrics: 0.6538)
Proc

,sensitivity,specificity,accuracy,precision,mcc,micro_auroc
mean,0.569890,0.992026,0.970160,0.796708,0.659140,0.915818
std,0.018709,0.000978,0.000657,0.015694,0.008899,0.008376


In [3]:
# for valid_csv, test_csv in zip(valid_csv_fliles, test_csv_fliles):
#     valid_df = aggregate_pred_dataframe([valid_csv])
#     test_df = aggregate_pred_dataframe([test_csv])
#     me = generate_mean_ensemble_metrics_auto(valid_df, test_df)
#     agg = pd.concat([agg, pd.DataFrame([me])], ignore_index=True)


# agg.aggregate(['mean', 'std'])

In [4]:
import pandas as pd

def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"

# Sample loading of your CSV - this would be replaced with your actual loading
result_df = pd.read_csv('result_cv/result_cv.csv')

# metrics = ['mcc', 'sensitivity', 'specificity', 'accuracy', 'precision', 'micro_auroc']
metrics = ['mcc', 'sensitivity', 'precision', 'micro_auroc']
grouped = result_df.groupby('model_key').agg({
    k: ['mean', 'std'] for k in metrics
})

record_counts = result_df.groupby('model_key').size()

for metric in metrics:
    grouped[f'{metric}_formatted'] = grouped.apply(lambda row: format_mean_std(row, metric), axis=1)


# Add the record count column to the grouped dataframe
grouped['record_count'] = record_counts.values

formatted_df = grouped[[k + '_formatted' for k in metrics] + ['record_count']]
formatted_df.columns = metrics + ['record_count']
formatted_df = formatted_df.sort_values(metrics[0], ascending=True)

formatted_df = formatted_df.reset_index()
formatted_df

,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,gearnet,0.453 ± 0.023,0.392 ± 0.053,0.584 ± 0.063,0.860 ± 0.011,5
1,bert,0.499 ± 0.010,0.432 ± 0.033,0.630 ± 0.051,0.892 ± 0.012,5
2,bert-gearnet,0.519 ± 0.031,0.521 ± 0.033,0.568 ± 0.062,0.905 ± 0.016,5
3,bert-gearnet-ensemble,0.578 ± 0.018,0.517 ± 0.041,0.691 ± 0.021,0.913 ± 0.006,5
4,esm-33-gearnet-ensemble-rus,0.656 ± 0.019,0.747 ± 0.016,0.612 ± 0.044,0.945 ± 0.004,5
5,esm-t33,0.664 ± 0.012,0.616 ± 0.028,0.749 ± 0.018,0.928 ± 0.011,10
6,esm-33-gearnet,0.676 ± 0.011,0.627 ± 0.019,0.762 ± 0.022,0.924 ± 0.008,10
7,esm-t33-ensemble,0.684 ± 0.002,0.672 ± 0.016,0.729 ± 0.018,0.952 ± 0.003,5
8,esm-t33-resiboost,0.689 ± 0.005,0.673 ± 0.036,0.738 ± 0.027,0.949 ± 0.004,4
9,esm-33-gearnet-resiboost-n25,0.690 ± 0.009,0.666 ± 0.043,0.749 ± 0.033,0.931 ± 0.006,5


In [5]:
from tabulate import tabulate

def visualize(formatted_df, metrics):
    markdown_table = tabulate(formatted_df[['model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
    print(markdown_table)

visualize(formatted_df=formatted_df, metrics=metrics)

| model_key                        | mcc           | sensitivity   | precision     | micro_auroc   |
|:---------------------------------|:--------------|:--------------|:--------------|:--------------|
| gearnet                          | 0.453 ± 0.023 | 0.392 ± 0.053 | 0.584 ± 0.063 | 0.860 ± 0.011 |
| bert                             | 0.499 ± 0.010 | 0.432 ± 0.033 | 0.630 ± 0.051 | 0.892 ± 0.012 |
| bert-gearnet                     | 0.519 ± 0.031 | 0.521 ± 0.033 | 0.568 ± 0.062 | 0.905 ± 0.016 |
| bert-gearnet-ensemble            | 0.578 ± 0.018 | 0.517 ± 0.041 | 0.691 ± 0.021 | 0.913 ± 0.006 |
| esm-33-gearnet-ensemble-rus      | 0.656 ± 0.019 | 0.747 ± 0.016 | 0.612 ± 0.044 | 0.945 ± 0.004 |
| esm-t33                          | 0.664 ± 0.012 | 0.616 ± 0.028 | 0.749 ± 0.018 | 0.928 ± 0.011 |
| esm-33-gearnet                   | 0.676 ± 0.011 | 0.627 ± 0.019 | 0.762 ± 0.022 | 0.924 ± 0.008 |
| esm-t33-ensemble                 | 0.684 ± 0.002 | 0.672 ± 0.016 | 0.729 ± 0.018 | 0.952 

In [6]:
target_df = formatted_df[formatted_df['model_key'].str.match('esm-33-gearnet-\d+') | formatted_df['model_key'].str.match('esm-t33-\d+')]

visualize(formatted_df=target_df, metrics=metrics)

| model_key   | mcc   | sensitivity   | precision   | micro_auroc   |
|-------------|-------|---------------|-------------|---------------|
